# RADMC-3D Volume Rendering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from disklab import radmc3d
from disklab.natconst import au
import radmc3d_volume_rendering as rvr

%matplotlib inline

## Setup

If we couldn't locate the radmc-3d sources, we need to set the path manually:

In [ ]:
rvr.locate_src_dir(dir='~/CODES/radmc-3d/version_0.41/src/')

We need to set a folder into which to write the output:

In [ ]:
dir_setup = 'radmc3d_setup'

Next, we write the files that are needed to compile RADMC-3D

In [ ]:
rvr.write_sources(dir_setup)

We also write the input data. This is stored in a file that we can access directly. Otherwise a dictionary can be given with the same contents.

In [ ]:
rvr.radmc3d_setup('fargo3d_data.npz', path=dir_setup)

## Compiling and running

Everything should be there now, so we need to compile:

In [ ]:
rvr.make(dir_setup)

In [ ]:
rvr.callit(path=dir_setup,
           command='image lambda 10 sizeradian 2 posang 90 projection 1 locobsau 0 0 .5 pointau 0 -1 0 nofluxcons npix 400'
          )

rvr.plotit(dir_setup)

## Debugging
For debugging: try reading the radmc3d input (note that the 
densities are not necessarily physical units)

#### Vertical/radial slice

In [ ]:
g = radmc3d.read_amr_grid(directory=dir_setup)
d = radmc3d.read_data(gas=True, mirror=False, ddir=dir_setup)

radmc3d.plotgrid(r=g.r / au, theta=g.theta, ri=g.ri / au, thetai=g.thetai, data=np.log10(d.rhog[:,:,:,0].min(-1)))
plt.ylim(-0.2,0.2)
plt.xlim(0.6,1.5);

#### Radial/azimuthal slice

In [ ]:
g = radmc3d.read_amr_grid(directory=dir_setup)
d = radmc3d.read_data(gas=True, mirror=False, ddir=dir_setup)

R,T = np.meshgrid(g.ri, g.phii, indexing='ij')

X = R * np.cos(T)
Y = R * np.sin(T)

f, ax = plt.subplots()
ax.pcolormesh(X / au, Y / au, np.log10(d.rhog[:,:,:,0].sum(1)))
ax.set_aspect('equal')

We can also make RADMC-3D write the vtk_grid and data

In [ ]:
!cd {dir_setup} && radmc3d vtk_grid
!cd {dir_setup} && radmc3d vtk_gas_density